In [66]:
import geopandas as gpd
from pathlib import Path
from pyproj import CRS
import shapely

In [77]:
streets_fp = Path("../Data/NVDB/NVDB_Vagtrafiknat.shp")        # your street network (SHP)
streets = gpd.read_file(streets_fp)
# streets = streets[["geometry", "Nättyp"]].rename(columns={"Nättyp": "Naettyp"})

bikes_fp   = Path("../Data/Cykelstrak_Linje.gpkg")  # your bike network (GPKG)
bikes   = gpd.read_file(bikes_fp)



In [70]:
# --- 0) Quick sanity prints (run once) ---
print("Streets CRS:", streets.crs)
print("Bikes   CRS:", bikes.crs)
print("Streets bounds:", streets.total_bounds)  
print("Bikes   bounds:", bikes.total_bounds)


# --- 1) Make valid + drop empties ---


# --- 2) Force identical CRS token (EPSG:3011, meters) ---
streets = streets.to_crs(3011) if streets.crs is None or streets.crs.to_epsg() != 3011 else streets.set_crs(3011, allow_override=True)
bikes   = bikes.to_crs(3011)   if bikes.crs   is None or bikes.crs.to_epsg()   != 3011 else bikes.set_crs(3011, allow_override=True)


Streets CRS: PROJCS["SWEREF99 18 00",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["central_meridian",18],PARAMETER["latitude_of_origin",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",150000],PARAMETER["false_northing",0],UNIT["m",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3011"]]
Bikes   CRS: EPSG:3011
Streets bounds: [ 102339.94985639 6390448.167       237112.56661416 6682532.421     ]
Bikes   bounds: [ 117470.14952275 6531742.1483303   171942.10915987 6618472.71671621]


In [71]:
print("Streets CRS:",streets.crs)
print("Bikes CRS:", bikes.crs)

streets = streets[~streets.geometry.is_empty & streets.geometry.notna()]
bikes   = bikes[~bikes.geometry.is_empty & bikes.geometry.notna()]

# Normalize CRS to the same EPSG token (no coordinate change if already 3011)
crs3011 = CRS.from_epsg(3011)

streets = streets.set_crs(crs3011, allow_override=True)
bikes   = bikes.set_crs(crs3011, allow_override=True)

print("Streets CRS:",streets.crs)
print("Bikes CRS:", bikes.crs)



Streets CRS: EPSG:3011
Bikes CRS: EPSG:3011
Streets CRS: EPSG:3011
Bikes CRS: EPSG:3011


In [72]:
# ==== BUFFER BIKE LINES (10 m) ====
bike_buffer_m = 10
# one corridor polygon for faster intersections
bike_corridor = bikes.buffer(bike_buffer_m).unary_union

# ==== 50% OVERLAP RULE ====
def overlap_ratio(street_geom, corridor):
    inter = street_geom.intersection(corridor)
    # handle empty intersections and zero-length segments safely
    L = max(street_geom.length, 1e-9)
    return 0.0 if inter.is_empty else (inter.length / L)

# optional quick prefilter so we don't compute intersections for far-away streets
near_mask = streets.geometry.distance(bike_corridor) <= bike_buffer_m

streets["Bike"] = 0
streets.loc[near_mask, "Bike"] = (
    streets.loc[near_mask, "geometry"]
           .apply(lambda g: 1 if overlap_ratio(g, bike_corridor) >= 0.5 else 0)
).astype(int)


/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: overflow encountered in distance
  return lib.distance(a, b, **kwargs)


In [80]:
print(streets.geometry.apply(lambda g: g is not None and hasattr(g, "has_z") and g.has_z).value_counts())


True     178891
False     25908
Name: count, dtype: int64


In [74]:
#  change crs
out_fp     = Path("../Data/Street_Network_categorized_final.shp")
streets.to_file(out_fp)
print(f"Wrote {out_fp.resolve()} with column 'Bike' (0/1).")

Wrote /Users/leonardhaas/Documents/Studium/Kurse/Web_and_Mobile_GIS/04_Project/Data/Street_Network_categorized_final.shp with column 'Bike' (0/1).
